# Projeto RAG - Ordem Paranormal

Alunos:

- Bruno Santos (bfss@cesar.school)
- Caio Guedes (ccsg@cesar.school)

# Escolha do documento

O documento escolhido foram duas páginas web de uma wiki comunitária que trata sobre o sistema brasileiro de RPG Ordem Paranormal. No universo do sistema, existe a nossa realidade e o chamado Outro Lado, um universo paranormal com criaturas e entidades que tentam invadir o nosso mundo.

Os documentos são duas páginas da wiki que falam a respeito do Ocultismo (principal perícia do sistema, que trata sobre as características de coisas do outro lado) e os Elementos do Outro Lado (tratando sobre os 5 principais elementos de criaturas, rituais e itens paranormais: Sangue, Morte, Energia, Conhecimento e Medo).

# Importações

In [20]:
!pip install langchain langchain_openai langchain_community

In [21]:
!pip install faiss-gpu

In [22]:
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Configuração chave API

Utilização de chave API própria da Open AI

In [23]:
 import getpass
 import os

 os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


# Carregando Documentos - web

In [24]:
# Lista de URLs das páginas da Wiki
urls = [
    "https://ordemparanormal.fandom.com/wiki/Ocultismo",
    "https://ordemparanormal.fandom.com/wiki/Elementos_do_Outro_Lado"
]

# Nome do arquivo
output_file = "ordem_paranormal.txt"

# Abrir o arquivo no modo de escrita
with open(output_file, "w", encoding="utf-8") as file:
    for url in urls:
        try:
            # Fazer a requisição e extrair o conteúdo
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extrair o texto da página
            content = soup.get_text()

            # Escrever o título da página como um separador
            file.write(f"\n\n--- Conteúdo da página: {url} ---\n\n")
            file.write(content)

            print(f"Conteúdo da URL {url} salvo com sucesso!")
        except Exception as e:
            print(f"Erro ao processar a URL {url}: {e}")

print(f"Todo o conteúdo foi salvo em {output_file}.")

Conteúdo da URL https://ordemparanormal.fandom.com/wiki/Ocultismo salvo com sucesso!
Conteúdo da URL https://ordemparanormal.fandom.com/wiki/Elementos_do_Outro_Lado salvo com sucesso!
Todo o conteúdo foi salvo em ordem_paranormal.txt.


O arquivo foi salvo em txt para posteriores consultas.

In [25]:
# Carregar o arquivo .txt
loader = TextLoader("ordem_paranormal.txt")
docs = loader.load()

print(f"Documento carregado com {len(docs)} página.")

Documento carregado com 1 página.


# Dividindo Documentos - Splitting/Chunking

O chunk_size foi definido em 1000 caracteres, herdando 200 do anterior. Foram realizados alguns testes utilizando valores de 1500 e 2000 caracteres porém o de 1000 gerou respostas mais eficazes e que respondem melhor as perguntas realizadas.

In [41]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(f"Splits: {len(all_splits)}.")

Splits: 87.


## Indexando - Store

In [42]:
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [43]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

## Teste Retriver

In [44]:
retrieved_docs = retriever.invoke("o que é o outro lado?")

len(retrieved_docs)

6

In [45]:
print(retrieved_docs[0].page_content)

O Outro Lado[]
O Outro Lado, no universo de Ordem Paranormal, é uma dimensão alternativa habitada por entidades insanas e misteriosas, também conhecidas como "elementos". A nossa realidade é separada do Outro Lado por uma barreira hipotética chamada Membrana. As duas dimensões se conectam naturalmente, com uma fazendo contato com a outra e vice-versa, porém, eventos macabros capazes de enfraquecer a Membrana através do Medo das pessoas podem fazer com que criaturas bizarras consigam invadir nossa realidade para gerar caos e destruição, aumentando a influência do paranormal e determinados locais.


# Buscando e Recuperando Informações - Retrieve

In [31]:
system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [32]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


# Gerando Respostas - Generate

In [33]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [34]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

## Perguntas e Respostas

Perguntas realizadas via código para teste:

In [35]:
for chunk in rag_chain.stream("O que é o Outro Lado?"):
    print(chunk, end="", flush=True)

O Outro Lado é uma dimensão alternativa no universo de Ordem Paranormal, habitada por entidades insanas, conhecida como "elementos". Ela se conecta à nossa realidade através de uma barreira chamada Membrana, a qual pode ser enfraquecida pelo Medo, permitindo a manifestação dessas entidades em nosso mundo.

In [49]:
for chunk in rag_chain.stream("Quais os elementos do outro lado?"):
    print(chunk, end="", flush=True)

Os cinco elementos do Outro Lado são Sangue, Morte, Conhecimento, Energia e Medo, cada um com propriedades únicas que influenciam suas manifestações na nossa realidade. As Relíquias da Calamidade atuam como a ponte entre esses elementos e a Realidade.

In [47]:
for chunk in rag_chain.stream("Me fale sobre as interações entre elementos"):
    print(chunk, end="", flush=True)

As interações entre elementos podem resultar em combinações que criam rituais ou criaturas, como o Ritual de Ouvir os Sussurros, que reúne Morte, Conhecimento e Energia. Elementos opostos, como Sangue, Morte, Conhecimento e Energia, podem se unir de maneira imprevisível sob a influência dos Seis Sinos de Tenebris, criando manifestações que desafiam a lógica da realidade.

In [48]:
for chunk in rag_chain.stream("O que são os sinos de Tenebris?"):
    print(chunk, end="", flush=True)

Os sinos de Tenebris representam a "união impossível daquilo que se repele", equilibrando os elementos Sangue, Morte, Conhecimento e Energia, que normalmente agem de forma contrária. Essa intersecção resulta na criação de criaturas e manifestações que desafiam a lógica e a realidade.

Todas as respostas acima estão condizentes com o contexto e o universo tratado no sistema de RPG.

Segmento para input de pergunta própria:

In [46]:
pergunta = input("Digite sua pergunta: ")
for chunk in rag_chain.stream(pergunta):
    print(chunk, end="", flush=True)

Digite sua pergunta: Quais as características do elemento Morte?
O elemento Morte é caracterizado por sua conexão com a distorção temporal, a percepção egóica da existência e a representação por espirais e Lodo Preto. Criaturas associadas a este elemento são apáticas, com movimentos inconstantes, e sua afinidade pode resultar em alterações físicas e psíquicas nos indivíduos.